<a href="https://colab.research.google.com/github/k7sung/clap2choir/blob/master/virtual_choir_tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title Choir folder location

input_dir = '' #@param {type:"string"}
conductor_video_path = "Default" #@param {type:"string"}



In [ ]:
#@title Parameters for Video Layout
VIDEO_RES = "854x480" #@param ["1920x1080","1280x720", "854x480", "540x360"] 
TITLE_H = 0 #@param {type:"integer"}
CAP_H = 0 #@param {type:"integer"} #120
MAR_W = 3

In [ ]:
#@title Parameters for downscaling
RESIZE_TO = 240 #@param {type:"integer"}
EXTRACT_AUDIO = True #@#param {type:"boolean"}


## codes

#### codes for align

In [ ]:

import os
from os.path import join as pathjoin
import librosa
import numpy as np
from librosa.display import waveplot
import matplotlib.pyplot as plt
import sklearn
from itertools import combinations as comb 
import scipy
from scipy.ndimage.morphology import grey_closing as closing
from os.path import basename, join as pathjoin
import ipywidgets as widgets
from ipywidgets import interact, Dropdown, FloatSlider, Checkbox
!pip install ffmpeg-python
import ffmpeg
import IPython.display

def is_conductingvideo(filename):
    if filename == conductor_video_path:
        return True
    lbname = basename(filename).lower()
    return ('master' in lbname) or ('conducting' in lbname) or ('conductor' in lbname)

def is_right_file(fname):
    fname_low = fname.lower()
    if "-handbrake" in fname_low:
        return False
    if fname_low.startswith("_"):
        return False
    for ext in [".gslides", ".pdf", ".jpg", ".png", ".gdoc"]:
        if fname_low.endswith(ext):
            return False
    return True


def list_files(d):
    for fname in os.listdir(d):
        fpath = pathjoin(d, fname)
        if os.path.isdir(fpath):
            continue
        if is_right_file(fname):
            yield fpath

def list_dirs(d):
    for fname in os.listdir(d):
        fpath = pathjoin(d, fname)
        if os.path.isdir(fpath):
            yield fpath

def get_all_files():
    for d in [input_dir]:#list_dirs(input_dir):
        yield from list_files(d)
    # for fname in list_dirs(input_dir):
    #     d = pathjoin(input_dir,fname)
    #     if os.path.isdir(d):
    #         yield from list_files(d)
    #     yield fname

def get_files():
    return get_all_files()
    # for f in get_all_files():
    #     if not is_conductingvideo(f):
    #         yield f

def get_conductor_video():
    if conductor_video_path.lower() != "default":
        return conductor_video_path
    for f in get_all_files():
        if is_conductingvideo(f):
            return f

def get_downscale_path(input_dir):
    return pathjoin(input_dir, "edits")

def get_align_data(tracks):
    align_to = max([t.get_adjusted_shift_sec() for t in tracks])
    to_trim = [(t.path, round(align_to - t.get_adjusted_shift_sec(), 3)) for t in tracks if not t.exclude]
    align_data = {}
    align_data["downscaled_dir"] = get_downscale_path(input_dir)
    align_data["file_ss"] = to_trim
    return align_data

conductor_video_path = get_conductor_video()

In [ ]:

SR = 22050
N_FFT = 1024
STFT_HLEN=4
def get_spec(sample):
    sample = sample/np.max(sample)
    sample_f = librosa.core.stft(sample, n_fft=N_FFT, hop_length=N_FFT//STFT_HLEN)
    sample_f = np.abs(sample_f)
    _,chan_idxes = (0, list(range(0,256)))
#     channels = sample_f[chan_idxes,:]
    channels = sample_f[:,:]
    channels = channels/np.max(channels)
    channels = np.log(channels+0.000000001)
#     channels_symbols = np.array([np.mean(channels[:, i:i + SYMBOL_WINDOW], axis=1) for i in range(0, channels.shape[1], SYMBOL_WINDOW//SYMBOL_SAMP)])
    return channels
# %time y_spec = get_sample_channels(y)
# chsums = channels
# plt.imshow(y_spec, aspect='auto')



def get_loud_regular_patterns(tops, expected_n=4, expected_interval=0):
    ts = tops[:,0] #[t for t, v in tops]
    ls = tops[:,1] #[v for t, v in tops]
    combs = [np.array(c) for c in comb(range(len(ts)), expected_n)]
    candidates = []
    for cidx in combs:
        intervals = np.diff(ts[cidx])
        int_mean = np.mean(intervals)
        int_deviation = abs(int_mean - expected_interval)
        int_var = np.var(intervals)
        loudness = np.sum(ls[cidx])
        candidates.append([int_deviation, int_var, loudness])
    candidates = np.array(candidates)
    max_dev, max_var, max_loud = candidates.max(axis=0)
    if expected_interval == 0:
        w = [0        , 1/max_var, 0.01/max_loud]
    else:
        w = [1/max_dev, 1/max_var, 0.01/max_loud]
    w = np.array(w)
    candidate_scores = candidates.dot(w)
    n=5
    bottom_n_idxs = candidate_scores.argpartition(n)[:n]
    # with np.printoptions(precision=3, suppress=True):
    # print("regulars", candidates[bottom_n_idxs])
    least_of_n_idx = candidate_scores[bottom_n_idxs].argmin()
    bottom_idx = bottom_n_idxs[least_of_n_idx]
    # print("most regular", candidates[bottom_idx])
    # debug_idx=[3,4,5,6]
    # for k in candidates:
    #     if np.all(k[0] == debug_idx):
    #         print("val of", k)
    #         break
    return ts[combs[bottom_idx]]
    # max_var = max([c[0] for c in candi_vars])
    # candi_vars = [(c[0] / max_var, c[1]) for c in candi_vars]
    # max_loud = max([c[0] for c in candi_louds])
    # candi_louds = [(-c[0] / max_loud, c[1]) for c in candi_louds]

    #minimize interval deviation,  variance, maximize loudness
    # scores = [(1.0 * v[0] + 0.01 * l[0], v[1]) for v, l in zip(candi_vars, candi_louds)]
    # # most_regular = min(scores)
    # scores.sort()
    # print("regulars:", scores[:5])
    # most_regular = scores[0]
    # print("most regular", most_regular)
    # return [ts[t] for t in most_regular[1]]




def load_sample(fname, duration=None):
    print("loading ", fname, SR)
    sample=None
    import warnings
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        sample, _ = librosa.load(fname, sr=SR, mono=True, duration=duration) 
        # sample = sample[:SR*60]
    return sample

def plot_claps(c, tops):
    fname = basename(c.path)
    print("\n" + fname + "\n")
    tops = np.array(tops)
    # p1 = plt.subplot(211)
    # plt.plot(c.t, c.V, '-', color="grey")
    # plt.plot(c.t, c.dV, '-', color="blue")
    plt.plot(c.t, 0.05*c.V + 1.0*c.dV, '-', color="blue")
    plt.plot(tops[:, 0], tops[:, 1], "x", color='black')
    # p2 = plt.subplot(212)
    # plt.plot(peak_ts, d_peaks, '-', color='blue')
    plt.plot(c.click_secs, [0] * len(c.click_secs), "o", color="red")
    # plt.ylim(ymin=-50)
    plt.show()

def get_window(win_size_ms):
    win_sample = librosa.time_to_frames(win_size_ms*0.001, 
        sr=SR, hop_length=N_FFT//STFT_HLEN)
    mid = win_sample//2
    # x_dist = [((x-mid)*2/win_sample) for x in range(win_sample)]
    # window = [1/(((x-mid)*6/win_sample)**2+0.5) for x in range(win_sample)]
    window = 1/(((np.arange(win_sample)-mid)*6/win_sample)**2+0.5)
    # plt.plot(x_dist)
    # plt.plot(window)
    return window

window = get_window(1200)
# window = get_window(600)

def impulses_from_meta(meta):
    imp_4 = np.zeros(meta.spec.shape[1])
    imp_idx = librosa.time_to_frames(
        meta.click_secs, 
        sr=SR, hop_length=N_FFT//STFT_HLEN
    )
    imp_4[imp_idx] = 1.0
    return imp_4

def impulses_from_tops(meta, expected_interval=0.8, top_n=60):
    expected_dist = int(expected_interval*SR / (N_FFT/STFT_HLEN)) * 0.5
    tops = meta.get_tops(top_n, expected_dist)
    imps = np.zeros(meta.spec.shape[1])
    idxes = librosa.time_to_frames(tops[:,0], sr=SR, hop_length=N_FFT//STFT_HLEN)
    imps[idxes] = tops[:,1]
    return imps

def get_master_blurred(master_meta):
    master_4 = impulses_from_meta(master_meta)
    # master_4 = impulses_from_envelope(master_meta.V)
    master_4_blurred = np.correlate(master_4, window, 'same')
    return master_4_blurred

def plot_fine_align(master_meta, signal_meta):
    fine_align(master_meta, [signal_meta], make_plot=True)
    plt.figure()
    # plt.subplot(2,1,1)
    plt.title(signal_meta.name)
    # plt.plot(t,c)
    # plt.plot(librosa.frames_to_time(
    #     range(-len(master_4),len(signal_4)-1), hop_length=N_FFT//STFT_HLEN
    #     ),
    #     corr)
    # ax = plt.subplot(2,1,2)
    # print(t_shift, maxidx)
    ax = plt.gca()
    ax.plot(master_meta.t, master_meta.V, color="blue")
    ax.plot(signal_meta.t+(master_meta.start_sec-signal_meta.start_sec), signal_meta.V, color="red")
    ax.plot(signal_meta.t+signal_meta.shift_sec, signal_meta.V, color="green")
    ax.set_xlim(xmax=15)

def fine_align(master_meta, signal_meta_arr, make_plot=False): #
    master_4 = get_master_blurred(master_meta)
    metrics = []
    for i, signal_meta in enumerate(signal_meta_arr):
        if signal_meta.path == master_meta.path:
            continue
        # signal_4 = impulses_from_meta(signal_meta)
        signal_4 = impulses_from_tops(signal_meta)
        corr = np.correlate(master_4, signal_4, "full")
        maxidx = np.argmax(corr)
        score = corr[maxidx]/np.std(corr)
        metrics.append([i, corr[maxidx], score])
        dt = librosa.frames_to_time(1, hop_length=N_FFT//STFT_HLEN)
        t_shift = dt*(maxidx-len(signal_4))
        if make_plot:
            plt.figure()
            plt.plot(librosa.frames_to_time(range(-len(signal_4), len(signal_4)-1), hop_length=N_FFT//STFT_HLEN), corr)
            plt.title(f'{signal_meta.name}, {round(corr[maxidx]/np.std(corr),3)}')
        if signal_meta.is_master:
            signal_meta.shift_sec = 0
        else:
            signal_meta.shift_sec = round(t_shift, 3)
    metrics = np.array(metrics)
    plt.figure()
    plt.title("clap detection")
    plt.plot(metrics[:,1], metrics[:,2], 'o')
    plt.xlim(left=0)
    plt.ylim(bottom=0)
    plt.ylabel("consistency")
    plt.xlabel("volume")
    for i in metrics:
        plt.annotate(int(i[0]), (i[1], i[2]))



class VoiceClip:
    # FRAME_SIZE = int(SR * 0.01)
    # HOP_LEN = FRAME_SIZE//2
    
    def __init__(self, path):
        quiet_threshold = -2400
        self.name = basename(path)
        self.path = path
#         print(VoiceClip.SR)
        wav = load_sample(path, duration=30)
        self.spec = get_spec(wav)
        self.click_secs = []
        self.start_sec = 0 #time when clapping ends
        self.shift_sec = 0 #time to start the video related to the master. negative means video should be trimmed at the beginning
        self.click_intervals = []
        self.is_master = is_conductingvideo(path)
        self.t = librosa.frames_to_time(range(self.spec.shape[1]), 
                                   sr=SR, hop_length=N_FFT//STFT_HLEN,
                                  )
        self.V = np.sum(self.spec[N_FFT//4:,:], axis=0)
        self.dV = np.diff(self.V, prepend=0)
        self.V = np.max([self.V, [quiet_threshold]*len(self.V)], axis=0)
        self.dV = np.diff(self.V, prepend=self.V[0])
        self.to_plot = None
        self.exclude = False
        self.offset = 0.0
    
    def get_adjusted_shift_sec(self):
        if self.exclude:
            return 0.0
        return self.shift_sec + self.offset

#         wav, _ = librosa.load(path, mono=True, sr=VoiceClip.SR, duration=12)
#         wav = VoiceClip.normalize(wav)
#         wav = librosa.effects.preemphasis(wav)
#         wav = librosa.effects.percussive()
#         self.ae, self.rms, self.dae = VoiceClip.get_time_features(wav)
    def get_tops(self, top_n, expected_dist=50):
        # print("finding top every", expected_dist)
        # w = lambda v, dv: 0.0 * v + 1.0 * dv
        v, dv = self.V, self.dV
        wV = 0.05 * v + 1.0 * dv
        peak_idxs, _ = scipy.signal.find_peaks(wV, distance=expected_dist)
        peak_ts = self.t[peak_idxs]
        peak_vals = wV[peak_idxs]

        tops = sorted([(t, w) for t, w 
                    in zip(peak_ts, peak_vals)], 
                    key=lambda p: p[1], 
                    reverse=True)[:top_n]
        tops = np.array(sorted(tops))

        # loudness_all = peak_vals
        # loudness_avg = np.mean(loudness_all)
        # loudness_std = np.std(loudness_all)
        # # loudness_top = np.mean(tops[:,1])
        # loudness_top = np.mean(sorted(tops[:,1]))#[len(tops)//2:])
        # # loudness_top = np.mean(sorted(tops[:,1])[len(tops)//2:])
        # # print(round(loudness_top,2))
        # # for i in tops[:,1]:
        # #     print(round(i,2)," ",end="")
        # # print()
        # # print("selected candidate loudness", round(candidates[bottom_idx, 2],3),
        # print("(loudness_top - loudness_avg)", (loudness_top - loudness_avg))
        # if (loudness_top - loudness_avg) < 1080:
        #     print("claps might not be present:")
        #     print("selected candidates loudness avg", round(loudness_top,3),
        #         "\nloudness avg, std:", round(loudness_avg,3), round(loudness_std,3),
        #         "\nperc of std:", 
        #         # round((candidates[bottom_idx, 2] - loudness_avg)/loudness_std, 3)*100)
        #         round((loudness_top - loudness_avg)/loudness_std*100, 3),
        #         "\ntop loudness difference", 
        #         round((loudness_top - loudness_avg), 3),
        #         )
        # # print("tops: ", tops)
        # # plt.plot(peak_ts, peak_vals)
        return tops

    def estimate_start(self, expected_interval = 0):
        # hpass_vol = np.sum(self.spec[N_FFT//4:,:], axis=0)
        # self.hapss_vol = hpass_vol
        # dVol = np.diff(self.V, prepend=0)
        # self.dV = dVol
        # peak_idxs, _ = scipy.signal.find_peaks(self.dVol, distance=50)
        # peak_ts = self.t[peak_idxs]
        # peak_vals = self.dVol[peaks]
        if expected_interval == 0:
            expected_interval= 0.375 #160bpm, really fast just to be sure
            expected_dist = int(expected_interval*SR / (N_FFT/STFT_HLEN))
        else:
            expected_dist = expected_interval / (self.t[1] - self.t[0]) * 0.8
        if self.is_master:
            tops = self.get_tops(12, expected_dist=expected_dist)
            clap_times = get_loud_regular_patterns(tops, 8, 
                expected_interval=expected_interval)[-4:]
        else:
            tops = self.get_tops(12, expected_dist=expected_dist)
            clap_times = get_loud_regular_patterns(tops, 
                expected_interval=expected_interval)
        # self.click_secs = self.t[clap_frames]
        self.click_secs = clap_times
        self.click_intervals = np.diff(self.click_secs)
        est_intv = np.mean(self.click_intervals)
        self.start_sec = np.mean(self.click_secs) + est_intv * 4.4
#         start_est = np.mean(clap_frames) + est_intv * 2
#         if is_finetune == False:
#             start_est = start_est - est_intv * 4.4
        # print("start est:", start_est)
        self.to_plot = lambda: plot_claps(self, tops)
        return self.start_sec
    
    def plot_claps(self):
        self.to_plot()



#### widgets

In [ ]:
#prepare alignment widget 
def update_track_idx(track_widget, offset_widget, exclude_widget, *args): #update calls this first
    # track_idx = track_widget.value
    track = tracks[track_widget.value]
    # offset_widget.value = offsets[track_idx]
    offset_widget.value = track.offset
    # exclude_widget.value = excludes[track_idx]
    exclude_widget.value = track.exclude

def update_exclude(track_widget, exclude_widget, *args):
    # excludes[track_widget.value] = exclude_widget.value
    tracks[track_widget.value].exclude = exclude_widget.value

def adjust_offset(track_idx, offset, exclude): #update calls this second, sometimes 3rd
    plt.plot(master.t, master.V)
    track = tracks[track_idx]
    # plt.plot(tracks[track_idx].t + tracks[track_idx].shift_sec + offset, tracks[track_idx].V)
    # plt.title(tracks[track_idx].name)
    plt.plot(track.t + track.shift_sec + offset, track.V)
    plt.title(track.name)
    plt.xlim([master.click_secs[0]-3,master.click_secs[-1]+3])
    # offsets[track_idx] = offset
    track.offset = offset

def prepare_manual_align_widget(tracks):
    track_widget = Dropdown(options=[(f"{i} {n}",i) for i, n in enumerate([t.name for t in tracks])])
    offset_widget = FloatSlider(min=-5.0, max=5.0, step=0.05, continuous_update=False)
    exclude_widget = Checkbox(False, description="do not align")
    track_widget.observe(
        lambda *args: update_track_idx(track_widget, offset_widget, exclude_widget, *args), "value")
    exclude_widget.observe(
        lambda *args: update_exclude(track_widget, exclude_widget, *args), "value")
    interact(adjust_offset, track_idx=track_widget, offset=offset_widget, exclude=exclude_widget);


In [ ]:
class mix_widget:
    def __init__(self, align_data):
        self.tracks = get_aligned_clips_path(align_data, ["audio_only"])
        self.track_widget = None
        self.channel_widget = None
        self.vol_widgets = []
        self.chans_lbl = ["ch 1", "ch 2", "ch 3", "ch 4", "piano", "mute"]
        self.chans_vol = [1.0, 1.0, 1.0, 1.0, 1.0, 0.0]
        self.tracks_to_channels=[0]*len(self.tracks)
        self.track_widget = Dropdown(options=
                [(f"{i} {n}",i) for i, n in enumerate([basename(p) for p in self.tracks])]
            )
        self.audio_widget = widgets.Audio()
        # self.chans = {"ch 1":0, "ch 2":1, "ch 3":2, "ch 4":3,
        #               "conductor":4, "mute":5}
        self.channel_widget = Dropdown(options=[(v,k) for k,v in enumerate(self.chans_lbl)])
        self.track_widget.observe(
            lambda *args:self._track_idx_updated(), "value")
        self.channel_widget.observe(
            lambda *args:self._channel_idx_updated(), "value")
        for lvl, lbl in zip(self.chans_vol, self.chans_lbl):
            vol_slider = FloatSlider(value=lvl, min=0, max=1, step=0.05, 
                description = lbl, orientation="vertical")
            if lbl == "mute":
                vol_slider.disabled = True
            vol_slider.observe(lambda *args:self._channel_vol_updated(),"value")
            self.vol_widgets.append(vol_slider)
        
    def _track_idx_updated(self):
        track_idx = self.track_widget.value
        self.channel_widget.value = self.tracks_to_channels[track_idx]
    
    def _channel_idx_updated(self):
        track_idx = self.track_widget.value
        self.tracks_to_channels[track_idx] = self.channel_widget.value

    def _channel_vol_updated(self):
        for i in range(len(self.chans_vol)):
            self.chans_vol[i] = self.vol_widgets[i].value

    def get_channels_to_tracks(self):
        # from collections import defaultdict
        # chs = defaultdict(list)
        ch2tracks={}
        mute_idx = 5
        for t, ch in enumerate(self.tracks_to_channels):
            if ch == mute_idx:
                continue
            ch2tracks.setdefault(ch,[]).append(self.tracks[t])
        return ch2tracks

    def mix(self):
        # if not os.path.exists("medium_room.wav"):
        #     !curl -LJO "https://github.com/k7sung/clap2choir/blob/master/medium_room.wav?raw=true"        
        ch2tracks = self.get_channels_to_tracks().items()
        # inputs = [ffmpeg.input(t.path) for t in self.tracks]
        # channels = [ffmpeg]
        channels = []
        for ch_idx, ch_tracks in ch2tracks:
            vol = self.chans_vol[ch_idx]
            inputs = [ffmpeg.input(t_path).audio.filter("loudnorm") for t_path in ch_tracks]
            f = (ffmpeg
                 .filter(inputs,"amix", inputs=len(inputs), duration="longest")
                 .filter("volume", vol) )
            channels.append(f)
        mixed = (ffmpeg.filter(channels, "amix", inputs=len(channels), duration="longest")
                    .filter("aecho",in_gain=1.0, out_gain=1.0, delays="15|25|40", decays="0.5|0.4|0.3")
                   ) #-af aecho=1.0:0.7:20:0.5
        # reverb = ffmpeg.input("medium_room.wav")
        # out_ch = ffmpeg.filter([mixed, reverb], "afir", gtype="none")
        out_ch = mixed
        return out_ch
        
    def display(self):
        # button = widgets.Button(description="preview")
        # def play_audio(*args):
        #     print(args)
        #     render_audio(m.mix(), "test.m4a",ss=10,t=30)
        #     self.audio_widget.from_file("test.m4a")
        #     # self.audio_widget.reload()
        # button.on_click(play_audio)
        return widgets.VBox(
                [widgets.HBox([self.track_widget, self.channel_widget]),
                 widgets.HBox(self.vol_widgets),
                #  button,
                #  self.audio_widget
                 ]
            )


#### codes for downscaling

In [ ]:
def has_convert_error(ret_vals):
    for t in ret_vals:
        if "Impossible to convert between the formats" in t:
            return True
        if "Unsupported input format:" in t:
            return True
    return False

def is_audio(fpath):
    fname = basename(fpath).lower()
    for ext in [".wav", ".mp3", ".m4a"]:
        if fname.endswith(ext):
            return True
    return False

def get_video_height(fpath):
    h = !ffprobe -loglevel quiet -select_streams v:0 -show_entries stream=height -of csv=p=0 "$fpath"
    return int(h[0])

def extract_clips(align_data, audio_only=False):
    if not audio_only and not os.path.exists("colab-ffmpeg-cuda"):
        !git clone https://github.com/rokibulislaam/colab-ffmpeg-cuda.git
        !cp -r ./colab-ffmpeg-cuda/bin/. /usr/bin/
        !ffmpeg -version

    output_dir = align_data["downscaled_dir"]
    file_ss = align_data["file_ss"]
    !mkdir -p "$output_dir"
    !mkdir -p "$output_dir"/"audio_only"
    print(len(file_ss), "files to process")
    i=0
    rpixels=RESIZE_TO
    bps ="0.5M"
    hw_cnt, sw_cnt, bypass_cnt = 0,0,0
    for f, t in file_ss:
        t = round(t,3)
        ddir = pathjoin(output_dir, basename(f))
        bfname = basename(f)
        adir = pathjoin(output_dir, "audio_only", f"{bfname}")
        print("file#",i, bfname, ddir)
        i+=1 


        if is_audio(f):
            print("audio only...")
            !ffmpeg -y -ss "$t" -i "$f" -c:a copy -loglevel error "$adir" 
            # !ffmpeg -y -ss "$t" -i "$f" -af loudnorm -y -loglevel error "$adir" 
            continue

        if EXTRACT_AUDIO:
            print("extracting audio...")
            !ffmpeg -y -ss "$t" -i "$f" -c:a copy -vn -loglevel error "$adir"

        if audio_only:
            continue

        if "invisible" in bfname.lower():
            print('video name has "invisible", skip...')
            bypass_cnt += 1
            continue

        if os.path.exists(ddir):
            print("video already processed...")
            bypass_cnt += 1
            continue

        if get_video_height(f) <= rpixels:
            bypass_cnt += 1
            print("trim video only...")
            !ffmpeg -y -ss "$t" -i "$f" -c copy -loglevel error "$ddir" 
            continue

        hw_cnt+=1
        ret_vals = !ffmpeg -y -hwaccel cuda -hwaccel_output_format cuda \
            -ss "$t" -i "$f" -vf scale_npp=-2:"$rpixels" \
            -c:a copy -c:v h264_nvenc \
            -b:v "$bps" -vsync 0 \
            -y -loglevel error "$ddir"
        if has_convert_error(ret_vals):
            print("plain scale_npp failed, use hwupload")
            ret_vals = !ffmpeg -y -hwaccel cuda -hwaccel_output_format cuda \
            -ss "$t" -i "$f" -vf "hwupload_cuda,scale_npp=w=-2:h=""$rpixels" \
            -c:a copy -c:v h264_nvenc \
            -b:v "$bps" -vsync 0 \
            -y -loglevel error "$ddir"
        if has_convert_error(ret_vals):
            hw_cnt-=1
            print("no luck, use default ffmpeg scaler")
            !ffmpeg -y -ss "$t" -i "$f" -vf scale=-2:"$rpixels" \
            -c:a copy -c:v libx264 -crf 20 -vsync 0 \
            -y -loglevel error "$ddir"
            # scale_ffmpeg(f, ddir, t, -2, rpixels)
            sw_cnt+=1

    print("GPU processed ",hw_cnt, "bypassed ", bypass_cnt, "others", sw_cnt)   


#### codes for compiling final video

In [ ]:
W, H = [int(t) for t in VIDEO_RES.split("x")]
background_color = (0,0,0)#(255,255,255)
H = H - TITLE_H - CAP_H

!pip install ffmpeg-python
import moviepy
# from moviepy.audio.AudioClip import AudioArrayClip
from moviepy.editor import VideoFileClip, CompositeVideoClip, clips_array

def rotate_vid(vid):
    if vid.rotation in [-90, 90, 270]:
        print(basename(vid.filename), vid.rotation)
        vid = vid.resize(vid.size[::-1])
        vid.rotation = 0
    return vid

def rotate_vids(vids):
    for i, v in enumerate(vids):
        vids[i] = rotate_vid(v)

def scroll_vids(vids, rows):
    vids = [v.resize(height=H//rows) for v in vids]
    return [vids]

def make_srolling_row(vids, rows):
    row = clips_array(scroll_vids(vids,rows), bg_color=(0,0,0))
    # print("before", row.size)
    row = row.crop(y_center=row.h//2, height=H//rows)
    # print("after", row.size)
    return row

def get_scroller(scroll_time):
    # print("get scroller")
    return lambda get_frame, t: scroll(get_frame, t, scroll_time)

def scroll(get_frame, t, scroll_time):
    frame = get_frame(t)
    # print(frame.size, t)
    if frame.shape[1] > W:
        scroll_len = frame.shape[1] - W
        scroll_speed = scroll_len / scroll_time 
        scroll_pos = int(t*scroll_speed)
        frame_region = frame[:,scroll_pos:scroll_pos+W]
    if frame.shape[1] <= W:
        frame_region = frame
    # if frame.shape[1] < W:
    #     scroll_len = W - frame.shape[1]
    #     scroll_speed = scroll_len / scroll_time
    #     scroll_pos = int(t*scroll_speed)
    #     frame_region = 
    return frame_region

# modifiedClip = my_clip.fl( scroll )

def make_scrolling_scene(vids, t_start, t_end, *boundary):
    clips = [vids[start:end] for start, end in zip(boundary[:-1], boundary[1:])]
    arr = [make_srolling_row(clip, len(clips)) for clip in clips]
    arr = [row.subclip(t_start,t_end).fl(get_scroller((t_end or row.duration)-t_start)) for row in arr]
    # if CAP_H:
    #     arr.append([caption_bar.set_duration(arr[0].duration)])
    return clips_array([[a] for a in arr], bg_color=background_color).set_start(t_start)#.crossfadein(0.2).crossfadeout(0.2)#.resize(width=W)


def crop_vid(vid, exp_W, exp_H):
    ar = vid.w/vid.h
    expAR = exp_W/exp_H
    if ar > expAR:
        #wider, need to trim x
        vid = vid.resize(height=exp_H-MAR_W)
        vid = vid.crop(width = exp_W-MAR_W, x_center=vid.w//2)
        # print(vid.size)
    else:
        #taller, need to trim y
        vid = vid.resize(width=exp_W-MAR_W)
        vid = vid.crop(height=exp_H-MAR_W, y_center=vid.h//2)
    return vid.margin(left=MAR_W, color=background_color)

def crop_vids(vids, rows):
    # vids = [v.crop(x_center=v.w // 2, width = max(W//len(vids), v.w)) for v in vids]
    vids = [crop_vid(v, (W-MAR_W)//len(vids), (H-MAR_W)//rows) for v in vids]
    # num,div = W-MAR_W, len(vids)
    # widths = [num//div + (1 if x<num % div else 0) for x in range(div)]
    # vids = [crop_vid(v, w, (H-MAR_W)//rows) for v,w in zip(vids, widths)]
    # print("each row", [v.size for v in vids])
    return [vids]

def make_row_vids(vids, rows):
    # row = clips_array(crop_vids(vids,rows), rows_widths=[W-MAR_W])
    row = clips_array(crop_vids(vids,rows))
    # print("before", row.size)
    # row = row.crop(y_center=row.h//2, height=((H-MAR_W)//rows))
    # print("after", row.size)
    return row.margin(top=MAR_W, color=background_color)

def make_stagger_scene(vids, t_start, t_end, *boundary):
    clips = [vids[start:end] for start, end in zip(boundary[:-1], boundary[1:])]
    arr = [[make_row_vids(clip, len(clips))] for clip in clips] #each row is only 1 column
    arr = [[row[0].subclip(t_start, t_end)] for row in arr]
    # if CAP_H:
    #     caption_bar = ColorClip((W,CAP_H), color=background_color)
    #     arr.append([caption_bar])
    arr = clips_array(arr, bg_color=background_color)
    arr_w,arr_h = arr.size
    # print(max(0, H-arr_h), max(0,W-arr_w))
    arr = arr.margin(
            right=max(0,W-arr_w), bottom=max(0, H-arr_h)+CAP_H, top=TITLE_H, color=background_color)#.resize(width=W)
    return arr.set_start(t_start)#.crossfadein(0.2).crossfadeout(0.2)


def dither(n, k): #evenly divide n elements into k groups of integers
    r0 = n/k
    inner, outer = 0, 0
    outs = []
    while outer < n:
        inner += r0
        outer_ = round(inner)
        out = outer_ - outer
        outs.append(out)
        outer = outer_
    return outs

def stagger(n, ratio=1.5):
    import math
    k = max(math.floor(math.sqrt(n/ratio)),1)
    return dither(n,k)

def layout_scene(all_vids):
    import itertools
    layout = stagger(len(all_vids))
    vids_in_rows = [0]+[i for i in layout]
    scene = make_stagger_scene(all_vids, 0, None, 
                                *itertools.accumulate(vids_in_rows))
    return scene

def render_audio(ff_track, fname, **kvargs):
    try:
        ff_track.output(fname, **kvargs).overwrite_output().run(capture_stdout=True, capture_stderr=True)
    except ffmpeg.Error as e:
        print(e.stderr.decode('utf8'))
        raise e

def get_aligned_clips_path(align_data, parts=[""]):
    hymn_dir = align_data["downscaled_dir"]
    # parts = [""] 
    vids={}
    for part in parts:
        part_dir = pathjoin(hymn_dir, part) 
        temp = !ls -1N "$part_dir"
        temp = [(pathjoin(hymn_dir, part, fname), fname) for fname in temp]
        vids[part] = [fpath for fpath,fname in temp if os.path.isfile(fpath)]
    # for k,v in vids.items():
    #     print(k, len(v), "vids")
    all_vids = []
    for p in parts:
        all_vids.extend(vids[p])
    # print("total", len(all_vids))
    return all_vids

def preview_video(align_data, arrangement=None):
    # OUTPUT_PATH = hymn_dir
    all_vids = [VideoFileClip(f) for f in get_aligned_clips_path(align_data)]
    if arrangement!=None:
        all_vids = [all_vids[i] for i in arrangement]
    
    rotate_vids(all_vids)
    scene = layout_scene(all_vids)
    return scene

def render_video(align_data, mixer, scene):
    output_path = pathjoin(align_data["downscaled_dir"], "output")
    !mkdir -p "$output_path"
    print("mixing audio...")
    audio_output_path = pathjoin(output_path,"output.m4a")
    render_audio(mixer.mix(), audio_output_path)
    t = CompositeVideoClip([scene])
    f_params=['-crf', '21']
    output_i = 0
    while os.path.exists(pathjoin(output_path, f"output{output_i}.mp4")):
        output_i += 1
    t.write_videofile(pathjoin(output_path, f"output{output_i}.mp4"), fps=25, 
            codec='libx264', 
            # audio_codec="aac", #audio=pathjoin(OUTPUT_PATH, "output.m4a"), 
            audio=audio_output_path,
            ffmpeg_params=f_params, threads=4)


## List of the videos to align:


In [ ]:
if conductor_video_path == None:
    print("Error!! Conductor video not found.")
else:
    print("conductor video:", basename(get_conductor_video()))
print("files to process:")
[(i, basename(f)) for i,f in enumerate(get_files())]

## Load & align the videos

In [ ]:
tracks  = [VoiceClip(fname) for fname in get_files()]
master= VoiceClip(conductor_video_path)
master.estimate_start()
fine_align(master, tracks)

### manual adjustment if needed...

In [ ]:
prepare_manual_align_widget(tracks)

## Mixer

In [ ]:
#every time this cell runs, it resets the mixer adjustments
align_data = get_align_data(tracks)
extract_clips(align_data, audio_only=True)
IPython.display.clear_output()
m = mix_widget(align_data)
m.display()

In [ ]:
render_audio(m.mix(), "preview.m4a",ss=30,t=15) #ss=start sec of preview; t=duration of preview
IPython.display.Audio('preview.m4a', autoplay=True)

## Extract the audio and video clips 


In [ ]:
extract_clips(align_data)

## Generate the choir video

In [ ]:
#preview
scene = preview_video(align_data)
moviepy.editor.ipython_display(scene, t=15, height=360)

In [ ]:
#render
render_video(align_data, m, scene)